# Extract all Market Pricing Data for Nifty 500 Stocks from Yahoo Finance

In [51]:
import pandas as pd
import yfinance as yf
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import ta as ta
from ta.trend import ADXIndicator
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import numpy as np
from pyfinance import ols
from datetime import date
import bt

# Read NIFTY 500 Symbols with Sector Codes and any Exclusions

In [65]:
import pandas as pd
read_nifty=pd.read_csv('NSE_500_Latest.csv')
equity_list_1=symbol_list=read_nifty['Symbol'].tolist()
read_nifty.head()
#read_nifty = read_nifty[read_nifty.Industry != 'FINANCIAL SERVICES']
#read_nifty = read_nifty[read_nifty.Industry != 'CONSTRUCTION']
#read_nifty = read_nifty[read_nifty.Industry != 'MEDIA & ENTERTAINMENT']

,Company Name,Industry,Symbol,Series,ISIN Code,Capitalization
0,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025,LARGE
1,Abbott India Ltd.,PHARMA,ABBOTINDIA,EQ,INE358A01014,LARGE
2,Adani Green Energy Ltd.,POWER,ADANIGREEN,EQ,INE364U01010,LARGE
3,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042,LARGE
4,Adani Transmission Ltd.,POWER,ADANITRANS,EQ,INE931S01010,LARGE


In [53]:
#Count of list of Equities
len(equity_list_1)

500

# Filter Dates for Data Extraction from Yahoo

In [54]:
FILTER_TIME_LOW = '2007-10-03'
FILTER_TIME_HIGH = '2020-12-13'

# Supporting Function for Technical Indicators if the need be

In [55]:
def yahoo_extractor(tick):
    data_yahoo=yf.download(tick, start=FILTER_TIME_LOW, end=FILTER_TIME_HIGH)
    #data_yahoo['Ticker']=tick
    #data_yahoo=data_yahoo.drop_duplicates()
    return data_yahoo
def CCI(data, ndays): 
    TP = (data['High'] + data['Low'] + data['Close']) / 3 
    CCI = pd.Series((TP - TP.rolling(ndays).mean()) / (0.015 * TP.rolling(ndays).std()),name = 'CCI') 
    #CCI=CCI.reset_index(level=2, drop=True)
    #data = data.join(CCI) 
    return CCI
def EMV(data, ndays): 
    dm = ((data['High'] + data['Low'])/2) - ((data['High'].shift(1) + data['Low'].shift(1))/2)
    br = (data['Volume'] / 100000000) / ((data['High'] - data['Low']))
    EMV = dm / br 
    EMV_MA = pd.Series(EVM.rolling(ndays).mean(), name = 'EVM')
    #EVM_MA=EVM_MA.reset_index(level=2, drop=True)
    return EMV_MA
def ROC(data,n):
    N = data['Close'].diff(n)
    D = data['Close'].shift(n)
    ROC = pd.Series(N/D,name='ROC')
    #ROC=ROC.reset_index(level=2, drop=True)
    return ROC 
def BBANDS(data, n):
    MA = data.Close.rolling(window=n).mean()
    SD = data.Close.rolling(window=n).std()
    UpperBB = MA + (2 * SD) 
    LowerBB = MA - (2 * SD)
    return UpperBB - LowerBB
def ForceIndex(data, ndays): 
    Force_Index = pd.Series(data['Close'].diff(ndays) * data['Volume'], name = 'ForceIndex') 
    #Force_Index=Force_Indexreset_index(level=2, drop=True)
    return Force_Index
def EWMA(data, s_days ,l_days): 
    EMA_s_days = pd.Series(data['Close'].ewm(span = s_days, min_periods = s_days - 1).mean(),name = 'EWMA_' + str(s_days)) 
    EMA_l_days = pd.Series(data['Close'].ewm(span = l_days, min_periods = l_days - 1).mean(),name = 'EWMA_' + str(l_days)) 
    EWMA_Signal=pd.Series(EMA_s_days-EMA_l_days,name='MACD_Signal')
    #EWMA_Signal=EWMA_Signal.reset_index(level=2, drop=True)
    return EWMA_Signal
def OBV(data):
    OBV=ta.volume.on_balance_volume(data['Close'], data['Volume'])
    #ta.volume.
    #OBV = ta.OBV()/10**6
    return OBV
def RSI(data):
    rsi=ta.momentum.rsi(data['Close'], n=14)
    #rsi = ta.RSI(data['Close'], timeperiod=14)
    return rsi
def ADX(data):
    data['Adj Open'] = data.Open * data['Adj Close']/data['Close']
    data['Adj High'] = data.High * data['Adj Close']/data['Close']
    data['Adj Low'] = data.Low * data['Adj Close']/data['Close']
    data.dropna(inplace=True)
    adx=ADXIndicator(data['Adj High'],data['Adj Low'],data['Adj Close'],n=14).adx()
    return adx
def calc_beta(x):
    print("Came inside here")
    benchmark=x['NSE500_Return'].values
    symbol_return=x['Return'].values
    beta=stats.linregress(benchmark,symbol_return)[1]
    return beta
def cum_return(x):
    #print(x.columns)
    z= pd.DataFrame((1 + x['Return'].rolling(250)).cumprod() - 1)-pd.DataFrame((1 + x['Return'].rolling(20)).cumprod() - 1)
    return z
beginning = '2004-06-01'
data_nse500 = bt.get('^CRSLDX', start=beginning)
data_nse500=data_nse500[data_nse500.index>=FILTER_TIME_LOW]
data_nse500=data_nse500[data_nse500.index<=FILTER_TIME_HIGH]
data_nse500.rename(columns = {'crsldx':'NSE500'}, inplace = True)
data_nse500['NSE500_Return']=(data_nse500['NSE500']/data_nse500['NSE500'].shift(1))-1

# Extract Data from Yahoo while filtering for Trading Volume, Penny Stocks etc. Also Calculate variants of momentum Factor

In [56]:
concat_df=pd.DataFrame()
count = 1
for symbol in equity_list_1:
    try:
        #print("Downloading data for Ticker", symbol)
        tick=symbol+".NS"
        dwn_dat = yahoo_extractor(tick)
        dwn_dat['Symbol']=symbol
        avg_vol=dwn_dat.tail(20)['Volume'].mean()
        price_filt=dwn_dat.tail(20)['Close'].mean()
        # Added Filters for INR Trading Volume. Can be removed if required
        if (avg_vol<=1000.0) or (price_filt< 20):
            print("Stock: ",symbol," is Delisted/Illiquid/ Under 50", price_filt, "  ",avg_vol )
        else:
            dwn_dat['Share_Turnover']=np.log(dwn_dat['Adj Close']*dwn_dat['Volume'])
            dwn_dat['Return']=np.log(dwn_dat['Close'] / dwn_dat['Close'].shift(1))
            dwn_dat['Momentum']=dwn_dat['Return'].rolling(250,min_periods=2).sum()-dwn_dat['Return'].rolling(21,min_periods=2).sum()
            dwn_dat['Mom_Sharpe']=dwn_dat['Return'].rolling(250).mean()/dwn_dat['Return'].rolling(250).std()
            dwn_dat['Mom_Gain_Loss']=(dwn_dat['Momentum']+(dwn_dat['Return'].rolling(250).agg(lambda x: (x > 0).count())/dwn_dat['Return'].rolling(250).agg(lambda x: (x < 0).count())))/2
            dwn_dat['Mom_Slope']=(dwn_dat['Momentum']+(dwn_dat['Momentum']-dwn_dat['Momentum'].shift(250)))/2 
            dwn_dat['StDev']=dwn_dat['Return'].rolling(250,min_periods=2).std()  
            dwn_dat['Liquidity']=(1/30)*np.log(dwn_dat['Volume'].rolling(30).sum()/dwn_dat['Share_Turnover'].rolling(30).sum())
            dwn_dat['Volatility']=dwn_dat['Return'].rolling(window=252).std() * np.sqrt(252)  
            dwn_dat=pd.concat([dwn_dat,data_nse500],axis=1)
            dwn_dat['NSE500_Return']=dwn_dat['NSE500_Return'].fillna(method='bfill')
            dwn_dat['NSE500_Return']=dwn_dat['NSE500_Return'].fillna(method='ffill')
            dwn_dat['Return']=dwn_dat['Return'].fillna(method='bfill')
            dwn_dat['Return']=dwn_dat['Return'].fillna(method='ffill')
            params=pd.DataFrame()
            rolling = ols.PandasRollingOLS(y=dwn_dat['Return'], x=dwn_dat['NSE500_Return'], window=252)
            params["CAPM_Beta"]=rolling.beta['feature1']
            params["CAPM_Alpha"]=rolling.alpha
            dwn_dat=dwn_dat.reset_index().merge(params.reset_index(),how='left',on='Date').set_index('Date')
            dwn_dat=dwn_dat.drop('NSE500_Return',axis=1)
            concat_df=concat_df.append(dwn_dat)
            print("Got data for ticker: ", "count: ", count, symbol)
            count=count +1
    except: 
        print("Ignoring Data for ticker: ", symbol)
concat_df=concat_df.reset_index()
concat_df.drop_duplicates(subset =['Date','Symbol'], keep = False, inplace = True) 
concat_df.set_index(['Date'], append=True, inplace=True)
concat_df.set_index(['Symbol'], append=True, inplace=True)
concat_df=concat_df.droplevel(0)
concat_df.index
concat_df=concat_df.reorder_levels(['Date', 'Symbol'])
concat_df.sort_index(inplace=True)
concat_df=concat_df.reset_index()
map_sectors=pd.read_csv("NSE_500_Latest.csv")
map_sectors=map_sectors[['Company Name','Industry','Symbol','Capitalization']]
df_merged = pd.merge(map_sectors, concat_df, on='Symbol')

[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  1 ACC
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  2 ABBOTINDIA
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  3 ADANIGREEN
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  4 ADANIPORTS
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  5 ADANITRANS
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  6 ALKEM
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  7 AMBUJACEM
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  8 ASIANPAINT
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  9 AUROPHARMA
[****

[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  73 PETRONET
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  74 PIDILITIND
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  75 PEL
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  76 PFC
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  77 POWERGRID
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  78 PGHH
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  79 PNB
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  80 RELIANCE
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  81 SBICARD
[*****************

[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  146 EXIDEIND
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  147 FEDERALBNK
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  148 FORTIS
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  149 FRETAIL
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  150 GMRINFRA
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  151 GILLETTE
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  152 GLAXO
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  153 GLENMARK
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  154 GODRE

[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  219 SUMICHEM
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  220 SUNTV
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  221 SUNDARMFIN
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  222 SUNDRMFAST
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  223 SUPREMEIND
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  224 SYMPHONY
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  225 SYNGENE
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  226 TTKPRESTIG
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  2

[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  287 CERA
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  288 CHALET
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  289 CHAMBLFERT
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  290 CHENNPETRO
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  291 COCHINSHIP
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  292 CYIENT
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  293 DBCORP
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  294 DCBBANK
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  295 DCMSHR

[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  355 INFIBEAM
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  356 INGERRAND
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  357 INOXLEISUR
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  358 JBCHEPHARM
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  359 JKLAKSHMI
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  360 JKPAPER
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  361 JKTYRE
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  362 JTEKTINDIA
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  

[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  427 RAIN
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  428 RALLIS
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  429 RCF
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  430 RATNAMANI
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  431 RAYMOND
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  432 REDINGTON
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  433 SCHNEIDER
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  434 SIS
[*********************100%***********************]  1 of 1 completed
Got data for ticker:  count:  435 SEQUENT
[******

In [57]:
df_merged.head()

,Company Name,Industry,Symbol,Capitalization,Date,Open,High,Low,Close,Adj Close,...,Momentum,Mom_Sharpe,Mom_Gain_Loss,Mom_Slope,StDev,Liquidity,Volatility,NSE500,CAPM_Beta,CAPM_Alpha
0,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,LARGE,2007-10-03,1225.800049,1261.300049,1175.050049,1228.550049,930.114319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4327.500000,NaN,NaN
1,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,LARGE,2007-10-04,1298.949951,1298.949951,1202.000000,1217.349976,921.634644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4335.250000,NaN,NaN
2,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,LARGE,2007-10-05,1222.000000,1227.699951,1161.099976,1177.949951,891.805786,...,0.0,NaN,NaN,NaN,0.016788,NaN,NaN,4301.750000,NaN,NaN
3,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,LARGE,2007-10-08,1177.949951,1230.000000,1097.000000,1115.150024,844.261169,...,0.0,NaN,NaN,NaN,0.022820,NaN,NaN,4195.250000,NaN,NaN
4,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,LARGE,2007-10-09,1084.099976,1184.250000,1084.099976,1176.250000,890.518921,...,0.0,NaN,NaN,NaN,0.046691,NaN,NaN,4369.700195,NaN,NaN


# Calculate Sector Momentum(Return) for each Stock based on last 1 month return

In [58]:
df_sector=pd.DataFrame(df_merged.groupby(['Date','Industry'])['Return'].mean()).reset_index()
df_sector[df_sector['Return']==""] = np.NaN
df_sector['Return']=df_sector.groupby('Industry')['Return'].fillna(method='bfill')
df_sector['Return']=df_sector.groupby('Industry')['Return'].fillna(method='ffill')
df_sector['Cum_Return']=df_sector.groupby('Industry')['Return'].apply(lambda x: np.cumprod(1 + x.tail(22)) - 1).droplevel(0)
df_sector['Cum_Return']=df_sector.groupby('Industry')['Cum_Return'].fillna(method='bfill')
df_sector['Cum_Return']=df_sector.groupby('Industry')['Cum_Return'].fillna(method='ffill')
df_sector['Ind_Rank']=df_sector.groupby('Date')['Cum_Return'].rank(ascending=1)
df_merged=df_merged.merge(df_sector[['Date','Industry','Ind_Rank']],how='left',on=['Date','Industry'])
df_merged['Sector_Return']=df_merged['Return']*df_merged['Ind_Rank']
#df_merged=df_merged.drop(['Ind_Rank'],axis=1)

# Dump Data to Data Folder

In [59]:
df_merged['Date']= pd.to_datetime(df_merged['Date'])
df_merged.to_csv('data/NSE_500_Output.csv')

# Sanity check on the data 

In [60]:
concat_df['Date'].max()

Timestamp('2020-12-11 00:00:00')

In [61]:
len(df_merged['Symbol'].unique().tolist())

483

In [63]:
df_merged.tail()

,Company Name,Industry,Symbol,Capitalization,Date,Open,High,Low,Close,Adj Close,...,Mom_Gain_Loss,Mom_Slope,StDev,Liquidity,Volatility,NSE500,CAPM_Beta,CAPM_Alpha,Ind_Rank,Sector_Return
1258124,eClerx Services Ltd.,IT,ECLERX,SMALLCAP,2020-12-07,785.000000,804.000000,785.0,787.799988,787.799988,...,0.650549,0.749816,0.034443,0.289059,0.544737,11070.250000,0.690578,0.001320,3.0,0.010682
1258125,eClerx Services Ltd.,IT,ECLERX,SMALLCAP,2020-12-08,794.700012,819.000000,770.0,774.500000,774.500000,...,0.688884,0.827271,0.034433,0.290994,0.545044,11088.650391,0.690273,0.001235,3.0,-0.051080
1258126,eClerx Services Ltd.,IT,ECLERX,SMALLCAP,2020-12-09,781.000000,822.049988,781.0,817.349976,817.349976,...,0.701558,0.853126,0.034538,0.293828,0.547396,11181.200195,0.694110,0.001457,4.0,0.215399
1258127,eClerx Services Ltd.,IT,ECLERX,SMALLCAP,2020-12-10,817.000000,817.000000,787.0,802.150024,802.150024,...,0.685575,0.817296,0.034392,0.294543,0.547296,11141.349609,0.693119,0.001452,3.0,-0.056315
1258128,eClerx Services Ltd.,IT,ECLERX,SMALLCAP,2020-12-11,799.049988,834.000000,799.0,808.900024,808.900024,...,0.670243,0.777174,0.034394,0.296259,0.546524,11171.400391,0.693052,0.001588,6.0,0.050278
